In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tqdm import tqdm
from tensorflow.keras.utils import Sequence
from scipy.ndimage import zoom

def get_normalized(vol):
    norm_volume = (vol-vol.min())/(vol.max()-vol.min())
    return norm_volume


def downsample_volume(volume, target_shape=(70, 90, 70)):
    # Calculate the zoom factors for each dimension
    zoom_factors = np.array(target_shape) / np.array(volume.shape)
    
    # Perform the downsampling
    downsampled_volume = zoom(volume, zoom_factors, order=1)  # order=1 for linear interpolation
    
    return downsampled_volume


def get_cropped_volume(val_sample):
    axial_sum = np.max(val_sample, axis=2)
    axial_msk = axial_sum>axial_sum.mean()
    coronal_sum = np.max(val_sample, axis=1)
    coronal_msk = coronal_sum>coronal_sum.mean()
    
    object_indices = np.argwhere(axial_msk)
    min_bounds = object_indices.min(axis=0)
    max_bounds = object_indices.max(axis=0) + 1  # +1 to include the max index
    x_min, x_max, y_min, y_max = min_bounds[0],max_bounds[0],min_bounds[1],max_bounds[1]

    object_indices = np.argwhere(coronal_msk)
    min_bounds = object_indices.min(axis=0)
    max_bounds = object_indices.max(axis=0) + 1  # +1 to include the max index
    # print(min_bounds[0],max_bounds[0],min_bounds[1],max_bounds[1])
    _, _, z_min, z_max = min_bounds[0],max_bounds[0],min_bounds[1],max_bounds[1]
    
    x_min, x_max = get_fix_cords(x_min, x_max, 152)
    y_min, y_max = get_fix_cords(y_min, y_max, 188)
    z_min, z_max = get_fix_cords(z_min, z_max, 144)
    cropped_volume = val_sample[x_min:x_max, y_min:y_max, z_min:z_max]
    cropped_volume = downsample_volume(cropped_volume, target_shape=(76, 94, 72))
    normalized_volume = get_normalized(cropped_volume)
    
    return normalized_volume
	
def get_fix_cords(xmn, xmx, fvalue):
    c_point = xmn + (xmx - xmn)/2
    nx_min = int(c_point-fvalue/2)
    nx_max = int(c_point+fvalue/2)
    return nx_min, nx_max

In [2]:
class DataGenerator(Sequence):
    def __init__(self, data_dir, participants, batch_size=8, dim=(76, 94, 72), shuffle=True):
        self.data_dir = data_dir
        self.participants = participants
        self.batch_size = batch_size
        self.dim = dim
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.participants))
        self.on_epoch_end()
        
    def __len__(self):
        # Denotes the number of batches per epoch
        return int(np.floor(len(self.participants) / self.batch_size))
    
    def __getitem__(self, index):
        # Generate one batch of data
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        participants_temp = [self.participants.iloc[k] for k in indexes]
        X, y = self.__data_generation(participants_temp)
        return X, y
    
    def on_epoch_end(self):
        # Updates indexes after each epoch
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def __data_generation(self, participants_temp):
        # Generates data containing batch_size samples
        X = np.empty((self.batch_size, *self.dim, 1))
        y = np.empty((self.batch_size), dtype=float)
        
        for i, participant in enumerate(participants_temp):
            file_path = os.path.join(self.data_dir, f'sub-{participant["participant_id"]}_preproc-quasiraw_T1w.npy')
            volume = np.squeeze(np.load(file_path))
            x = get_cropped_volume(volume)
            X[i,] = x[..., np.newaxis]
            y[i] = participant["age"]
        
        return X, y

In [4]:
import tensorflow.compat.v1 as tf
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from sklearn.model_selection import train_test_split
from ResNet import generateAgePredictionResNet
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.models import load_model
# Disable TensorFlow v2 behavior
tf.disable_v2_behavior()

# Paths
# data_dir = r'C:\Users\User\Documents\Dataset\Brain Age\val\val_quasiraw'
# tsv_path = r'C:\Users\User\Documents\Dataset\Brain Age\participants.tsv'
tsv_path = r'C:\Users\User\Documents\Dataset\Brain Age\train\train_labels\participants.tsv'
data_dir = r'C:\Users\User\Documents\Dataset\Brain Age\train\train_quasiraw'

# Load participants information
participants = pd.read_csv(tsv_path, sep='\t')

# Split the participants into training and validation sets
train_participants, val_participants = train_test_split(participants, test_size=0.2, random_state=42)

# Create data generators
train_generator = DataGenerator(data_dir, train_participants, batch_size=32, dim=(76, 94, 72), shuffle=True)
val_generator = DataGenerator(data_dir, val_participants, batch_size=32, dim=(76, 94, 72), shuffle=False)

# Define the model
input_shape = (76, 94, 72)
model = generateAgePredictionResNet(input_shape, paddingType='same', initType='he_uniform', regAmount=0.00005, dropRate=0.2, includeScannerGender=False)
# model_path = 'normalized_best_weights.h5'
# model = load_model(model_path)
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('./test_weights/3DResNet_best_weights.h5', monitor='val_loss', save_best_only=True, mode='min')
tensorboard = TensorBoard(log_dir='./test_weights/logs')

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=100,
    callbacks=[early_stopping, checkpoint, tensorboard]
)

# model = generateAgePredictionResNet(input_shape, paddingType='same', initType='he_uniform', regAmount=0.00005, dropRate=0.2, includeScannerGender=False)

# # Compile the model
# model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# # Define callbacks
# early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
# tensorboard = TensorBoard(log_dir='./logs')

# # Train the model
# history = model.fit(
#     train_generator,
#     validation_data=val_generator,
#     epochs=100,
#     callbacks=[early_stopping, tensorboard]
# )

# Save the trained model
model.save('./test_weights/3DResNet_final_weights.h5')

Instructions for updating:
Colocations handled automatically by placer.
Epoch 1/100
10/80 [==>...........................] - ETA: 2:43 - batch: 4.5000 - size: 32.0000 - loss: 2331.1381 - mean_absolute_error: 29.7395

KeyboardInterrupt: 

In [28]:
# Create data generators
train_generator = DataGenerator(data_dir, train_participants, batch_size=16, dim=(70, 90, 70), shuffle=True)
val_generator = DataGenerator(data_dir, val_participants, batch_size=16, dim=(70, 90, 70), shuffle=False)

# Define the model
input_shape = (70, 90, 70)
model = generateAgePredictionResNet(input_shape, paddingType='same', initType='he_uniform', regAmount=0.00005, dropRate=0.2, includeScannerGender=False)
model_path = 'age_prediction_model_baan.h5'
model = load_model(model_path)
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_weights.h5', monitor='val_loss', save_best_only=True, mode='min')
tensorboard = TensorBoard(log_dir='./logs')

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=100,
    callbacks=[early_stopping, checkpoint, tensorboard]
)

# Save the trained model
model.save('age_prediction_model_baan_n.h5')

Epoch 1/100
161/161 [==============================] - 314s 2s/step - batch: 80.0000 - size: 16.0000 - loss: 9358751.4113 - mean_absolute_error: 360.1777 - val_loss: 467130.7331 - val_mean_absolute_error: 116.9538
Epoch 2/100
161/161 [==============================] - 306s 2s/step - batch: 80.0000 - size: 16.0000 - loss: 1193370.0898 - mean_absolute_error: 137.2098 - val_loss: 84681.0109 - val_mean_absolute_error: 44.2465
Epoch 3/100
161/161 [==============================] - 302s 2s/step - batch: 80.0000 - size: 16.0000 - loss: 52331.9634 - mean_absolute_error: 33.5531 - val_loss: 7021.4397 - val_mean_absolute_error: 15.3906
Epoch 4/100
161/161 [==============================] - 302s 2s/step - batch: 80.0000 - size: 16.0000 - loss: 80696.9346 - mean_absolute_error: 39.0963 - val_loss: 5283.7132 - val_mean_absolute_error: 17.8752
Epoch 5/100
161/161 [==============================] - 300s 2s/step - batch: 80.0000 - size: 16.0000 - loss: 687801.0568 - mean_absolute_error: 63.1001 - val_

## Inference 

In [58]:
ages = np.array(ages)
pred_ages = np.array(pred_ages)
ages.shape, pred_ages.shape

((757,), (757, 1))

In [12]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr
from tensorflow.keras.models import load_model

def calculate_metrics(model_path, tsv_path, data_dir):
    # Load the model
    model = load_model(model_path)

    # Load participants data
    participants = pd.read_csv(tsv_path, sep='\t')

    # Initialize dictionaries to store ground-truth and predicted ages by sex
    ages = {'male': [], 'female': []}
    pred_ages = {'male': [], 'female': []}

    # Iterate through each participant
    for participant_id, age, sex in tqdm(zip(participants['participant_id'], participants['age'], participants['sex']), total=len(participants)):
        file_path = os.path.join(data_dir, f'sub-{participant_id}_preproc-quasiraw_T1w.npy')
        if os.path.exists(file_path):
            volume = np.squeeze(np.load(file_path))
            x = get_cropped_volume(volume)
            data = x[np.newaxis, ..., np.newaxis]
            pred_age = model.predict(data)
            pred_ages[sex].append(pred_age[0][0])
            ages[sex].append(age)

    # Function to calculate metrics
    def compute_metrics(ages, pred_ages):
        ages = np.array(ages)
        pred_ages = np.array(pred_ages)
        mae = mean_absolute_error(ages, pred_ages)
        sd = np.std(pred_ages - ages)
        bias = np.mean(pred_ages - ages)
        rmse = np.sqrt(mean_squared_error(ages, pred_ages))
        # corr, _ = pearsonr(ages, pred_ages)
        return mae, sd, bias, rmse, 

    # Calculate and print metrics for males
    print("Metrics for Males:")
    mae_m, sd_m, bias_m, rmse_m = compute_metrics(ages['male'], pred_ages['male'])
    print(f'MAE: {mae_m}')
    print(f'SD: {sd_m}')
    print(f'Bias: {bias_m}')
    print(f'RMSE: {rmse_m}')
    # print(f'Correlation Coefficient: {corr_m}')

    # Calculate and print metrics for females
    print("Metrics for Females:")
    mae_f, sd_f, bias_f, rmse_f = compute_metrics(ages['female'], pred_ages['female'])
    print(f'MAE: {mae_f}')
    print(f'SD: {sd_f}')
    print(f'Bias: {bias_f}')
    print(f'RMSE: {rmse_f}')
    # print(f'Correlation Coefficient: {corr_f}')

    return {
        'Males': {'MAE': mae_m, 'SD': sd_m, 'Bias': bias_m, 'RMSE': rmse_m},
        'Females': {'MAE': mae_f, 'SD': sd_f, 'Bias': bias_f, 'RMSE': rmse_f}
    }

# Example usage
model_path = 'normalized_age_prediction_model.h5'
tsv_path = r'C:\Users\User\Documents\Dataset\Brain Age\val\val_labels\participants.tsv'
data_dir = r'C:\Users\User\Documents\Dataset\Brain Age\val\val_quasiraw'
metrics = calculate_metrics(model_path, tsv_path, data_dir)
print(metrics)

  0%|                                                                                          | 0/757 [00:00<?, ?it/s]C:\Users\User\.conda\envs\tf_gpu\lib\site-packages\tensorflow\python\keras\engine\training.py:2464: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
100%|████████████████████████████████████████████████████████████████████████████████| 757/757 [01:42<00:00,  7.42it/s]

Metrics for Males:
MAE: 3.4708773127183035
SD: 4.951340085579843
Bias: -0.682629689207356
RMSE: 4.998174860452277
Metrics for Females:
MAE: 4.506732293088629
SD: 6.708578934044181
Bias: -2.448790777347428
RMSE: 7.1415409811554795
{'Males': {'MAE': 3.4708773127183035, 'SD': 4.951340085579843, 'Bias': -0.682629689207356, 'RMSE': 4.998174860452277}, 'Females': {'MAE': 4.506732293088629, 'SD': 6.708578934044181, 'Bias': -2.448790777347428, 'RMSE': 7.1415409811554795}}


In [10]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import load_model
from tqdm import tqdm 
from scipy.ndimage import zoom, rotate

def get_normalized(vol):
    norm_volume = (vol - vol.min()) / (vol.max() - vol.min())
    return norm_volume

def downsample_volume(volume, target_shape=(76, 94, 72)):
    zoom_factors = np.array(target_shape) / np.array(volume.shape)
    downsampled_volume = zoom(volume, zoom_factors, order=1)
    return downsampled_volume

def get_cropped_volume(val_sample):
    axial_sum = np.max(val_sample, axis=2)
    axial_msk = axial_sum > axial_sum.mean()
    coronal_sum = np.max(val_sample, axis=1)
    coronal_msk = coronal_sum > coronal_sum.mean()
    
    object_indices = np.argwhere(axial_msk)
    min_bounds = object_indices.min(axis=0)
    max_bounds = object_indices.max(axis=0) + 1
    x_min, x_max, y_min, y_max = min_bounds[0], max_bounds[0], min_bounds[1], max_bounds[1]

    object_indices = np.argwhere(coronal_msk)
    min_bounds = object_indices.min(axis=0)
    max_bounds = object_indices.max(axis=0) + 1
    _, _, z_min, z_max = min_bounds[0], max_bounds[0], min_bounds[1], max_bounds[1]
    
    x_min, x_max = get_fix_cords(x_min, x_max, 152)
    y_min, y_max = get_fix_cords(y_min, y_max, 188)
    z_min, z_max = get_fix_cords(z_min, z_max, 144)
    cropped_volume = val_sample[x_min:x_max, y_min:y_max, z_min:z_max]
    cropped_volume = downsample_volume(cropped_volume, target_shape=(76, 94, 72))
    normalized_volume = get_normalized(cropped_volume)
    
    return normalized_volume

def get_fix_cords(xmn, xmx, fvalue):
    c_point = xmn + (xmx - xmn) / 2
    nx_min = int(c_point - fvalue / 2)
    nx_max = int(c_point + fvalue / 2)
    return nx_min, nx_max

def preprocess_volume(volume):
    x = get_cropped_volume(volume)
    x = x[np.newaxis, ..., np.newaxis]  # Add batch and channel dimensions
    return x

def load_volume(file_path):
    volume = np.squeeze(np.load(file_path))
    return volume

def inference(model, file_path):
    volume = load_volume(file_path)
    processed_volume = preprocess_volume(volume)
    reconstruction, age_prediction, gender_prediction = model.predict({'Input': processed_volume})

    return reconstruction, age_prediction[0], gender_prediction[0]

def compute_metrics(true_values, pred_values):
    mse = mean_squared_error(true_values, pred_values)
    mae = mean_absolute_error(true_values, pred_values)
    rmse = np.sqrt(mse)
    return mse, mae, rmse

def compute_sd(values):
    return np.std(values)

# Load the model
model_path = './weights/normalized_age_prediction_model.h5'
model = load_model(model_path)

# Load participants information
data_dir = r'C:\Users\User\Documents\Dataset\Brain Age\val\val_quasiraw'
participants_tsv = r'C:\Users\User\Documents\Dataset\Brain Age\val\val_labels\participants.tsv'
participants = pd.read_csv(participants_tsv, sep='\t')

# Load the model
model = load_model(model_path)

# Load participants data
# participants = pd.read_csv(tsv_path, sep='\t')

# Initialize dictionaries to store ground-truth and predicted ages by sex
ages = {'male': [], 'female': []}
pred_ages = {'male': [], 'female': []}

# Initialize lists to store true and predicted values
true_ages = []
pred_ages = []
genders = []
# pred_genders = []

# Iterate through each participant
for participant_id, age, sex in tqdm(zip(participants['participant_id'], participants['age'], participants['sex']), total=len(participants)):
    file_path = os.path.join(data_dir, f'sub-{participant_id}_preproc-quasiraw_T1w.npy')
    if os.path.exists(file_path):
        volume = np.squeeze(np.load(file_path))
        x = get_cropped_volume(volume)
        data = x[np.newaxis, ..., np.newaxis]
        pred_age = model.predict(data)
        # pred_ages[sex].append(pred_age[0][0])
        pred_ages.append(pred_age[0][0])
        true_ages.append(age)
        genders.append(1 if sex == 'male' else 0)
        
            


# Perform inference on all samples
# for index, row in participants.iterrows():
#     participant_id = row['participant_id']
#     age = row['age']
#     gender = row['sex']  # 'male' or 'female'
#     file_path = os.path.join(data_dir, f'sub-{participant_id}_preproc-quasiraw_T1w.npy')

#     reconstruction, age_prediction, gender_prediction = inference(model, file_path)

#     true_ages.append(age)
#     pred_ages.append(age_prediction)
#     genders.append(1 if gender == 'male' else 0)
#     pred_genders.append(gender_prediction)
# Convert to numpy arrays
true_ages = np.array(true_ages)
pred_ages = np.array(pred_ages)
genders = np.array(genders)
# pred_genders = np.array(pred_genders)

# Compute metrics for overall samples
overall_mse, overall_mae, overall_rmse = compute_metrics(true_ages, pred_ages)
overall_mse_sd = compute_sd((true_ages - pred_ages)**2)
overall_mae_sd = compute_sd(np.abs(true_ages - pred_ages))
overall_rmse_sd = compute_sd(np.sqrt((true_ages - pred_ages)**2))

# Compute metrics for male samples
male_mask = genders == 1
male_mse, male_mae, male_rmse = compute_metrics(true_ages[male_mask], pred_ages[male_mask])
male_mse_sd = compute_sd((true_ages[male_mask] - pred_ages[male_mask])**2)
male_mae_sd = compute_sd(np.abs(true_ages[male_mask] - pred_ages[male_mask]))
male_rmse_sd = compute_sd(np.sqrt((true_ages[male_mask] - pred_ages[male_mask])**2))

# Compute metrics for female samples
female_mask = genders == 0 
female_mse, female_mae, female_rmse = compute_metrics(true_ages[female_mask], pred_ages[female_mask])
female_mse_sd = compute_sd((true_ages[female_mask] - pred_ages[female_mask])**2)
female_mae_sd = compute_sd(np.abs(true_ages[female_mask] - pred_ages[female_mask]))
female_rmse_sd = compute_sd(np.sqrt((true_ages[female_mask] - pred_ages[female_mask])**2))

# Print the results
print(f'Overall - MSE: {overall_mse:.2f} (SD: {overall_mse_sd:.2f}), MAE: {overall_mae:.2f} (SD: {overall_mae_sd:.2f}), RMSE: {overall_rmse:.2f} (SD: {overall_rmse_sd:.2f})')
print(f'Male - MSE: {male_mse:.2f} (SD: {male_mse_sd:.2f}), MAE: {male_mae:.2f} (SD: {male_mae_sd:.2f}), RMSE: {male_rmse:.2f} (SD: {male_rmse_sd:.2f})')
print(f'Female - MSE: {female_mse:.2f} (SD: {female_mse_sd:.2f}), MAE: {female_mae:.2f} (SD: {female_mae_sd:.2f}), RMSE: {female_rmse:.2f} (SD: {female_rmse_sd:.2f})')


  0%|                                                                                          | 0/757 [00:00<?, ?it/s]

1/1 [==============================] - 6s 6s/step


  0%|                                                                                | 1/757 [00:06<1:18:20,  6.22s/it]

1/1 [==============================] - 0s 14ms/step


  0%|▏                                                                                 | 2/757 [00:06<33:28,  2.66s/it]

...

1/1 [==============================] - 0s 13ms/step


100%|███████████████████████████████████████████████████████████████████████████████▉| 756/757 [01:52<00:00,  7.23it/s]

1/1 [==============================] - 0s 12ms/step


100%|████████████████████████████████████████████████████████████████████████████████| 757/757 [01:52<00:00,  6.75it/s]

Overall - MSE: 36.67 (SD: 106.56), MAE: 3.94 (SD: 4.60), RMSE: 6.06 (SD: 4.60)
Male - MSE: 24.98 (SD: 56.73), MAE: 3.47 (SD: 3.60), RMSE: 5.00 (SD: 3.60)
Female - MSE: 51.09 (SD: 144.95), MAE: 4.51 (SD: 5.55), RMSE: 7.15 (SD: 5.55)


In [11]:

# Print the results
print(f'Overall - MSE: {overall_mse:.2f} (SD: {overall_mse_sd:.2f}), MAE: {overall_mae:.2f} (SD: {overall_mae_sd:.2f}), RMSE: {overall_rmse:.2f} (SD: {overall_rmse_sd:.2f})')
print(f'Male - MSE: {male_mse:.2f} (SD: {male_mse_sd:.2f}), MAE: {male_mae:.2f} (SD: {male_mae_sd:.2f}), RMSE: {male_rmse:.2f} (SD: {male_rmse_sd:.2f})')
print(f'Female - MSE: {female_mse:.2f} (SD: {female_mse_sd:.2f}), MAE: {female_mae:.2f} (SD: {female_mae_sd:.2f}), RMSE: {female_rmse:.2f} (SD: {female_rmse_sd:.2f})')


Overall - MSE: 36.67 (SD: 106.56), MAE: 3.94 (SD: 4.60), RMSE: 6.06 (SD: 4.60)
Male - MSE: 24.98 (SD: 56.73), MAE: 3.47 (SD: 3.60), RMSE: 5.00 (SD: 3.60)
Female - MSE: 51.09 (SD: 144.95), MAE: 4.51 (SD: 5.55), RMSE: 7.15 (SD: 5.55)
